In [1]:
import pandas as pd

Prepare the economical data

In [2]:
raw_econ_df = pd.read_csv("data\Annual_FY_June2024.csv")

In [4]:
# raw_econ_df.interpolate(method='quadratic')

In [3]:
econ_df = raw_econ_df.ffill()
econ_df = raw_econ_df.bfill()
econ_df.isna().sum().sum()

np.int64(0)

In [4]:
# add the inflation rate
econ_df['previous_cpiu'] = econ_df['cpiu'].shift(1)
econ_df['inflation_rate'] = ((econ_df['cpiu'] - econ_df['previous_cpiu']) / econ_df['previous_cpiu']) * 100
econ_df.drop(columns=['previous_cpiu'], inplace=True)
econ_df = econ_df.bfill()

Prepare the interest rates

In [5]:
raw_interest_rates_df = pd.read_csv('data\FEDFUNDS.csv')
raw_interest_rates_df.isna().sum().sum()

np.int64(0)

In [6]:
raw_interest_rates_df['DATE'] = pd.to_datetime(raw_interest_rates_df['DATE'])
raw_interest_rates_df['DATE'] = raw_interest_rates_df['DATE'].dt.year

Prepare the data for the model

In [16]:
interest_df = raw_interest_rates_df[(raw_interest_rates_df['DATE'] >= 1955) & (raw_interest_rates_df['DATE'] <= 2023)].copy()
econ_df = econ_df[(econ_df['date'] >= 1955) & (econ_df['date'] <= 2023)]

In [19]:
interest_df.rename(columns={'DATE' : 'date', 'FEDFUNDS' : 'fedfunds'}, inplace=True)

In [20]:
data = pd.merge(econ_df, interest_df, on='date', how='inner')

Prepare the data for the LSTM